In [68]:
import os
from collections import Counter
import cv2
import matplotlib.pyplot as plt

dataset_dir = "dataset1"  # Replace with the actual path
train_dir = os.path.join(dataset_dir)

classes = os.listdir(train_dir)
class_counts = {}

for cls in classes:
    cls_path = os.path.join(train_dir, cls)
    images = os.listdir(cls_path)
    class_counts[cls] = len(images)

# Show count per class
for cls, count in sorted(class_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{cls}: {count}")


glass: 250
paper: 250
plastic: 250


In [69]:
def preprocess_image(img, size=(64, 64)):
    img_resized = cv2.resize(img, size)
    
    return img_resized

In [70]:
import numpy as np
from skimage.feature import hog
from skimage.color import rgb2gray

def extract_features(img):
    # Ensure image is grayscale for HOG
    gray = rgb2gray(img)

    # HOG features
    hog_features, _ = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                          orientations=9, block_norm='L2-Hys', visualize=True)

    # Color histogram (for each channel separately, 8 bins per channel as example)
    hist_r, _ = np.histogram(img[:, :, 0], bins=8, range=(0, 1), density=True)
    hist_g, _ = np.histogram(img[:, :, 1], bins=8, range=(0, 1), density=True)
    hist_b, _ = np.histogram(img[:, :, 2], bins=8, range=(0, 1), density=True)
    
    color_hist = np.concatenate([hist_r, hist_g, hist_b])

    # Combine HOG and color histogram
    combined_features = np.concatenate([hog_features, color_hist])

    return combined_features

In [71]:
import numpy as np
from tqdm import tqdm

X, y = [], []

for cls in classes:
    cls_path = os.path.join(train_dir, cls)
    for file in tqdm(os.listdir(cls_path)):  # Limit for performance
        img_path = os.path.join(cls_path, file)
        img = cv2.imread(img_path)
        img_pre = preprocess_image(img)
        feat = extract_features(img_pre)
        X.append(feat)
        y.append(cls)

X = np.array(X)
y = np.array(y)


  0%|          | 0/250 [00:00<?, ?it/s]c:\Users\ysfah\anaconda3\Lib\site-packages\numpy\lib\histograms.py:885: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
100%|██████████| 250/250 [00:02<00:00, 87.13it/s]


In [72]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Train/test split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

# Initialize and train Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[43  6  1]
 [ 1 47  2]
 [ 9  4 37]]
              precision    recall  f1-score   support

       glass       0.81      0.86      0.83        50
       paper       0.82      0.94      0.88        50
     plastic       0.93      0.74      0.82        50

    accuracy                           0.85       150
   macro avg       0.85      0.85      0.85       150
weighted avg       0.85      0.85      0.85       150



In [73]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def detect_objects_and_classify(original):
    # Load image


    # Contrast enhancement (CLAHE)
    lab = cv2.cvtColor(original, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    enhanced = cv2.merge((cl, a, b))
    enhanced_bgr = cv2.cvtColor(enhanced, cv2.COLOR_LAB2BGR)

    # Blur to reduce noise
    blurred = cv2.GaussianBlur(enhanced_bgr, (5, 5), 0)

    # Multi-channel edge detection
    b, g, r = cv2.split(blurred)
    edges_b = cv2.Canny(b, 50, 150)
    edges_g = cv2.Canny(g, 50, 150)
    edges_r = cv2.Canny(r, 50, 150)
    combined_edges = cv2.bitwise_or(edges_b, edges_g)
    combined_edges = cv2.bitwise_or(combined_edges, edges_r)

    # Optional: color segmentation for red objects
    hsv = cv2.cvtColor(original, cv2.COLOR_BGR2HSV)
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)

    # Combine edge and color masks
    final_mask = cv2.bitwise_or(combined_edges, red_mask)

    # Morphological operations to enhance edges
    kernel = np.ones((5, 5), np.uint8)
    final_mask = cv2.dilate(final_mask, kernel, iterations=2)
    final_mask = cv2.morphologyEx(final_mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(final_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        area = cv2.contourArea(c)
        aspect_ratio = w / float(h)
        if area > 100 and w > 30 and h > 30 and 0.2 < aspect_ratio < 5.0:
            roi = original[y:y+h, x:x+w]

            # Assumes these exist:
            roi_pre = preprocess_image(roi)  # Preprocessing function
            # seg = segment_image(roi_pre)
            feat = extract_features(roi_pre).reshape(1, -1)  # Feature extractor

            label = model.predict(feat)[0]  # ML model

            cv2.rectangle(original, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(original, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (36, 255, 12), 2)


    return original


In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=detect_objects_and_classify,
    inputs=gr.Image(type="numpy"), 
    outputs=gr.Image(type="numpy"),  
    live=True,
    title="Object Detection and Classification",
    description="Upload an image for object detection and classification."
)

# Launch the interface
iface.launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


c:\Users\ysfah\anaconda3\Lib\site-packages\numpy\lib\histograms.py:885: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
Traceback (most recent call last):
  File "c:\Users\ysfah\anaconda3\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ysfah\anaconda3\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ysfah\anaconda3\Lib\site-packages\gradio\blocks.py", line 2146, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ysfah\anaconda3\Lib\site-packages\gradio\blocks.py", line 1664, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 